In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from __future__ import print_function
from statsmodels.compat import lzip
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.linear_model import LinearRegression

C:\Users\sanka\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
from statsmodels.graphics.regressionplots import plot_fit
from statsmodels.graphics.regressionplots import plot_leverage_resid2
from statsmodels.graphics.regressionplots import plot_regress_exog
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.pipeline import make_pipeline

In [3]:
from sklearn.metrics import r2_score, mean_squared_error
linear_regression = linear_model.LinearRegression(normalize=False, fit_intercept=True)
def r2_est(X,y):
    return r2_score(y, linear_regression.fit(X,y).predict(X))
def r2_est_two(X,y, X_new, y_new):
    return r2_score(y_new, linear_regression.fit(X,y).predict(X_new))

In [4]:
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.cross_validation import cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import make_scorer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFECV
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import Lasso
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats import expon

C:\Users\sanka\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\sanka\Anaconda2\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [5]:
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
label_enc = LabelEncoder()
label_bin = LabelBinarizer()

## Data Loading and Preparation (time series)

In [6]:
cal = pd.read_csv('Datasources/inside_airbnb/calendar.csv')

In [7]:
cal['price'] = cal['price'].astype(str).map(lambda x: x.lstrip('$'))
cal['price'] = cal['price'].apply(pd.to_numeric, errors='coerce')
cal['price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [8]:
cal['date'] = pd.to_datetime(cal['date'])
cal['month'] = cal['date'].apply(lambda x: x.month)
cal['day'] = cal['date'].apply(lambda x: x.day)

In [9]:
cal['day_of_week'] = cal['date'].dt.weekday_name

In [10]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

In [11]:
cl = calendar()
holidays = cl.holidays(start=cal['date'].min(), end=cal['date'].max())

In [12]:
cal['holiday'] = cal['date'].isin(holidays)

In [13]:
c = cal.loc[cal.available!='f']
c = c[['listing_id','date','price','month','day_of_week','holiday']]

In [14]:
c_hol = c[c['holiday']==True]
c_wke = c[(c['holiday']==False)&((c['day_of_week']=='Sunday')|(c['day_of_week']=='Saturday'))]
c_wkd = c[(~c.isin(c_hol['date']))&(~c.isin(c_wke['date']))]

In [15]:
mean_price_hol = c_hol.groupby(by='listing_id')['price'].mean()
mean_price_wke = c_wke.groupby(by='listing_id')['price'].mean()
mean_price_wkd = c_wkd.groupby(by='listing_id')['price'].mean()

In [16]:
mean_price_hol = pd.DataFrame(mean_price_hol)
mean_price_wke = pd.DataFrame(mean_price_wke)
mean_price_wkd = pd.DataFrame(mean_price_wkd)

In [17]:
mean_price_hol = mean_price_hol.reset_index()
mean_price_wke = mean_price_wke.reset_index()
mean_price_wkd = mean_price_wkd.reset_index()

In [18]:
listings_withtopics = pd.read_csv('listings_withtopics.csv')
listings = pd.read_csv('Datasources/inside_airbnb/listings.csv')

C:\Users\sanka\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,1,2,3,4,5,6,10,11,12,13,15,17,18,20,21,22,54,62,65) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [19]:
def string_to_set(x):
    c = set()
    for w in x[1:-1].split(","):
        c.add(w)
        
    return c

In [20]:
listings_withtopics['amenities_set'] = listings_withtopics['amenities'].fillna('{}').map(string_to_set)

In [21]:
Topic_feature_list = []
for w in listings_withtopics.columns:
    if 'Topic' in w:
        Topic_feature_list.append(w)

In [22]:
df = pd.DataFrame()

for (index, rows) in listings_withtopics.iterrows():
    if index != 4927:
        df = df.append(rows)
        

In [23]:
df['id'] = df['id'].astype('int')

In [24]:
master_hol = df.merge(mean_price_hol, how='inner', left_on='id', right_on='listing_id')
master_wke = df.merge(mean_price_wke, how='inner', left_on='id', right_on='listing_id')
master_wkd = df.merge(mean_price_wkd, how='inner', left_on='id', right_on='listing_id')

## Feature Creation (time series)




### Features Created in regression

In [25]:
master_hol['host_response_rate'] = master_hol['host_response_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_hol['host_response_rate'] = master_hol['host_response_rate'].apply(pd.to_numeric, errors='coerce')
master_hol['host_response_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['host_acceptance_rate'] = master_hol['host_acceptance_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_hol['host_acceptance_rate'] = master_hol['host_acceptance_rate'].apply(pd.to_numeric, errors='coerce')
master_hol['host_acceptance_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['extra_people'] = master_hol['extra_people'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['extra_people'] = master_hol['extra_people'].apply(pd.to_numeric, errors='coerce')
master_hol['extra_people'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['weekly_price'] = master_hol['weekly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['weekly_price'] = master_hol['weekly_price'].apply(pd.to_numeric, errors='coerce')
master_hol['weekly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['monthly_price'] = master_hol['monthly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['monthly_price'] = master_hol['monthly_price'].apply(pd.to_numeric, errors='coerce')
master_hol['monthly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['security_deposit'] = master_hol['security_deposit'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['security_deposit'] = master_hol['security_deposit'].apply(pd.to_numeric, errors='coerce')
master_hol['security_deposit'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_hol['cleaning_fee'] = master_hol['cleaning_fee'].astype(str).map(lambda x: x.lstrip('$'))
master_hol['cleaning_fee'] = master_hol['cleaning_fee'].apply(pd.to_numeric, errors='coerce')
master_hol['cleaning_fee'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [26]:
master_wke['host_response_rate'] = master_wke['host_response_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_wke['host_response_rate'] = master_wke['host_response_rate'].apply(pd.to_numeric, errors='coerce')
master_wke['host_response_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['host_acceptance_rate'] = master_wke['host_acceptance_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_wke['host_acceptance_rate'] = master_wke['host_acceptance_rate'].apply(pd.to_numeric, errors='coerce')
master_wke['host_acceptance_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['extra_people'] = master_wke['extra_people'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['extra_people'] = master_wke['extra_people'].apply(pd.to_numeric, errors='coerce')
master_wke['extra_people'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['weekly_price'] = master_wke['weekly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['weekly_price'] = master_wke['weekly_price'].apply(pd.to_numeric, errors='coerce')
master_wke['weekly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['monthly_price'] = master_wke['monthly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['monthly_price'] = master_wke['monthly_price'].apply(pd.to_numeric, errors='coerce')
master_wke['monthly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['security_deposit'] = master_wke['security_deposit'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['security_deposit'] = master_wke['security_deposit'].apply(pd.to_numeric, errors='coerce')
master_wke['security_deposit'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wke['cleaning_fee'] = master_wke['cleaning_fee'].astype(str).map(lambda x: x.lstrip('$'))
master_wke['cleaning_fee'] = master_wke['cleaning_fee'].apply(pd.to_numeric, errors='coerce')
master_wke['cleaning_fee'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [27]:
master_wkd['host_response_rate'] = master_wkd['host_response_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_wkd['host_response_rate'] = master_wkd['host_response_rate'].apply(pd.to_numeric, errors='coerce')
master_wkd['host_response_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['host_acceptance_rate'] = master_wkd['host_acceptance_rate'].astype(str).map(lambda x: x.rstrip("%"))
master_wkd['host_acceptance_rate'] = master_wkd['host_acceptance_rate'].apply(pd.to_numeric, errors='coerce')
master_wkd['host_acceptance_rate'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['extra_people'] = master_wkd['extra_people'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['extra_people'] = master_wkd['extra_people'].apply(pd.to_numeric, errors='coerce')
master_wkd['extra_people'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['weekly_price'] = master_wkd['weekly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['weekly_price'] = master_wkd['weekly_price'].apply(pd.to_numeric, errors='coerce')
master_wkd['weekly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['monthly_price'] = master_wkd['monthly_price'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['monthly_price'] = master_wkd['monthly_price'].apply(pd.to_numeric, errors='coerce')
master_wkd['monthly_price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['security_deposit'] = master_wkd['security_deposit'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['security_deposit'] = master_wkd['security_deposit'].apply(pd.to_numeric, errors='coerce')
master_wkd['security_deposit'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
master_wkd['cleaning_fee'] = master_wkd['cleaning_fee'].astype(str).map(lambda x: x.lstrip('$'))
master_wkd['cleaning_fee'] = master_wkd['cleaning_fee'].apply(pd.to_numeric, errors='coerce')
master_wkd['cleaning_fee'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [28]:
master_hol['ones'] = np.ones(len(master_hol))
master_wke['ones'] = np.ones(len(master_wke))
master_wkd['ones'] = np.ones(len(master_wkd))

In [29]:
def has_amenity(x, amen_):
    if amen_ in x:
        return 1
    pass

In [30]:
all_amenities = set()

for idx in listings_withtopics['amenities'].fillna('{}').map(string_to_set).index:
    all_amenities = all_amenities.union(listings_withtopics['amenities'].fillna('{}').map(string_to_set)[idx])
    
all_amenities

{'',
 '"24-Hour Check-in"',
 '"Air Conditioning"',
 '"Buzzer/Wireless Intercom"',
 '"Cable TV"',
 '"Carbon Monoxide Detector"',
 '"Elevator in Building"',
 '"Family/Kid Friendly"',
 '"Fire Extinguisher"',
 '"First Aid Kit"',
 '"Free Parking on Premises"',
 '"Hair Dryer"',
 '"Hot Tub"',
 '"Indoor Fireplace"',
 '"Laptop Friendly Workspace"',
 '"Lock on Bedroom Door"',
 '"Other pet(s)"',
 '"Pets Allowed"',
 '"Pets live on this property"',
 '"Safety Card"',
 '"Smoke Detector"',
 '"Smoking Allowed"',
 '"Suitable for Events"',
 '"Wheelchair Accessible"',
 '"Wireless Internet"',
 '"translation missing: en.hosting_amenity_49"',
 '"translation missing: en.hosting_amenity_50"',
 '.89583333333',
 'Breakfast',
 'Cat(s)',
 'Dog(s)',
 'Doorman',
 'Dryer',
 'Essentials',
 'Gym',
 'Hangers',
 'Heating',
 'Internet',
 'Iron',
 'Kitchen',
 'Pool',
 'Shampoo',
 'TV',
 'Washer'}

In [31]:
for amen in all_amenities:
    master_hol['has' + amen] = 0
    master_wke['has' + amen] = 0
    master_wkd['has' + amen] = 0
    master_hol['has' + amen] = master_hol['amenities_set'].map(lambda x: has_amenity(x, amen))
    master_wke['has' + amen] = master_wke['amenities_set'].map(lambda x: has_amenity(x, amen))
    master_wkd['has' + amen] = master_wkd['amenities_set'].map(lambda x: has_amenity(x, amen))

In [32]:
has_amenties_list = []
for amen in all_amenities:
    has_amenties_list.append('has' + amen)

In [33]:
master_hol[has_amenties_list] = master_hol[has_amenties_list].fillna(0)
master_wke[has_amenties_list] = master_wke[has_amenties_list].fillna(0)
master_wkd[has_amenties_list] = master_wkd[has_amenties_list].fillna(0)

In [34]:
master_hol[Topic_feature_list] = master_hol[Topic_feature_list].astype('float')
master_wke[Topic_feature_list] = master_wke[Topic_feature_list].astype('float')
master_wkd[Topic_feature_list] = master_wkd[Topic_feature_list].astype('float')

In [35]:
X_hol = master_hol.select_dtypes(include=['float64', 'int64'])
X_wke = master_wke.select_dtypes(include=['float64', 'int64'])
X_wkd = master_wkd.select_dtypes(include=['float64', 'int64'])

In [36]:
X_hol.isnull().sum()

Topic0                                             0
Topic1                                             0
Topic10                                            0
Topic11                                            0
Topic12                                            0
Topic13                                            0
Topic14                                            0
Topic15                                            0
Topic16                                            0
Topic17                                            0
Topic18                                            0
Topic19                                            0
Topic2                                             0
Topic3                                             0
Topic4                                             0
Topic5                                             0
Topic6                                             0
Topic7                                             0
Topic8                                        

In [37]:
X_hol = X_hol.drop(['neighbourhood_group_cleansed','square_feet','has_availability','license','weekly_price','monthly_price'], axis = 1)
X_wke = X_wke.drop(['neighbourhood_group_cleansed','square_feet','has_availability','license','weekly_price','monthly_price'], axis = 1)
X_wkd = X_wkd.drop(['neighbourhood_group_cleansed','square_feet','has_availability','license','weekly_price','monthly_price'], axis = 1)

In [38]:
X_hol = X_hol.fillna(value=X_hol.mean())
X_wke = X_wke.fillna(value=X_wke.mean())
X_wkd = X_wkd.fillna(value=X_wkd.mean())

In [39]:
X_hol = X_hol.drop(['listing_id','host_id','scrape_id'], axis = 1)
X_wke = X_wke.drop(['listing_id','host_id','scrape_id'], axis = 1)
X_wkd = X_wkd.drop(['listing_id','host_id','scrape_id'], axis = 1)

In [40]:
X_hol['policy_enc'] = label_enc.fit_transform(master_hol.cancellation_policy)
master_hol['host_response_time'] = master_hol['host_response_time'].astype(str)
X_hol['host_resp_time_enc'] = label_enc.fit_transform(master_hol.host_response_time)
master_hol['calendar_updated'] = master_hol['calendar_updated'].astype(str)
X_hol['calendar_updated_enc'] = label_enc.fit_transform(master_hol.calendar_updated)
master_hol['bed_type'] = master_hol['bed_type'].astype(str)
X_hol['bed_type_enc'] = label_enc.fit_transform(master_hol.bed_type)
master_hol['jurisdiction_names'] = master_hol['jurisdiction_names'].astype(str)
X_hol['jurisdiction_names_enc'] = label_enc.fit_transform(master_hol.jurisdiction_names)
master_hol['zipcode'] = master_hol['zipcode'].astype(str)
X_hol['zipcode_enc'] = label_enc.fit_transform(master_hol.zipcode)

In [41]:
X_wke['policy_enc'] = label_enc.fit_transform(master_wke.cancellation_policy)
master_wke['host_response_time'] = master_wke['host_response_time'].astype(str)
X_wke['host_resp_time_enc'] = label_enc.fit_transform(master_wke.host_response_time)
master_wke['calendar_updated'] = master_wke['calendar_updated'].astype(str)
X_wke['calendar_updated_enc'] = label_enc.fit_transform(master_wke.calendar_updated)
master_wke['bed_type'] = master_wke['bed_type'].astype(str)
X_wke['bed_type_enc'] = label_enc.fit_transform(master_wke.bed_type)
master_wke['jurisdiction_names'] = master_wke['jurisdiction_names'].astype(str)
X_wke['jurisdiction_names_enc'] = label_enc.fit_transform(master_wke.jurisdiction_names)
master_wke['zipcode'] = master_wke['zipcode'].astype(str)
X_wke['zipcode_enc'] = label_enc.fit_transform(master_wke.zipcode)

In [42]:
X_wkd['policy_enc'] = label_enc.fit_transform(master_wkd.cancellation_policy)
master_wkd['host_response_time'] = master_wkd['host_response_time'].astype(str)
X_wkd['host_resp_time_enc'] = label_enc.fit_transform(master_wkd.host_response_time)
master_wkd['calendar_updated'] = master_wkd['calendar_updated'].astype(str)
X_wkd['calendar_updated_enc'] = label_enc.fit_transform(master_wkd.calendar_updated)
master_wkd['bed_type'] = master_wkd['bed_type'].astype(str)
X_wkd['bed_type_enc'] = label_enc.fit_transform(master_wkd.bed_type)
master_wkd['jurisdiction_names'] = master_wkd['jurisdiction_names'].astype(str)
X_wkd['jurisdiction_names_enc'] = label_enc.fit_transform(master_wkd.jurisdiction_names)
master_wkd['zipcode'] = master_wkd['zipcode'].astype(str)
X_wkd['zipcode_enc'] = label_enc.fit_transform(master_wkd.zipcode)

In [43]:
df_two = pd.DataFrame()

for (index, row) in master_hol.iterrows():
    if index != 3914:
        df_two = df_two.append(row)

In [44]:
master_hol = df_two

In [45]:
master_hol = master_hol.reset_index()

In [46]:
df_three = pd.DataFrame()

for (index, row) in X_hol.iterrows():
    if index != 3914:
        df_three = df_three.append(row)
        
X_hol = df_three

In [47]:
master_hol = master_hol.drop('index' , axis = 1)

In [48]:
X_hol.index = range(len(X_hol))

In [49]:
X_hol['host_is_superhost'] = label_bin.fit_transform(master_hol.host_is_superhost)
X_hol['is_location_exact'] = label_bin.fit_transform(master_hol.is_location_exact)
X_hol['host_profile_pic'] = label_bin.fit_transform(master_hol.host_has_profile_pic)
X_hol['host_identity_verified'] = label_bin.fit_transform(master_hol.host_identity_verified)
X_hol['instant_bookable'] = label_bin.fit_transform(master_hol.instant_bookable)
X_hol['require_guest_profile_picture'] = label_bin.fit_transform(master_hol.require_guest_profile_picture)
X_hol['require_guest_phone_verification'] = label_bin.fit_transform(master_hol.require_guest_phone_verification)

In [50]:
master_wke[(master_wke.host_is_superhost != 't') & (master_wke.host_is_superhost != 'f')]['host_is_superhost']

4400    NaN
Name: host_is_superhost, dtype: object

In [51]:
df_four = pd.DataFrame()

for (index, row) in master_wke.iterrows():
    if index != 4400:
        df_four = df_four.append(row)
        
master_wke = df_four
master_wke.index = range(len(master_wke))

In [52]:
df_five = pd.DataFrame()

for (index, row) in X_wke.iterrows():
    if index != 4400:
        df_five = df_five.append(row)
        
X_wke = df_five
X_wke.index = range(len(X_wke))

In [53]:
X_wke['host_is_superhost'] = label_bin.fit_transform(master_wke.host_is_superhost)
X_wke['is_location_exact'] = label_bin.fit_transform(master_wke.is_location_exact)
X_wke['host_profile_pic'] = label_bin.fit_transform(master_wke.host_has_profile_pic)
X_wke['host_identity_verified'] = label_bin.fit_transform(master_wke.host_identity_verified)
X_wke['instant_bookable'] = label_bin.fit_transform(master_wke.instant_bookable)
X_wke['require_guest_profile_picture'] = label_bin.fit_transform(master_wke.require_guest_profile_picture)
X_wke['require_guest_phone_verification'] = label_bin.fit_transform(master_wke.require_guest_phone_verification)

In [54]:
master_wkd[(master_wkd.host_is_superhost != 't') & (master_wkd.host_is_superhost != 'f')]['host_is_superhost']

4443    NaN
Name: host_is_superhost, dtype: object

In [55]:
df_six = pd.DataFrame()

for (index, row) in master_wkd.iterrows():
    if index != 4443:
        df_six = df_six.append(row)
        
master_wkd = df_six
master_wkd.index = range(len(master_wkd))

In [56]:
df_seven = pd.DataFrame()

for (index, row) in X_wkd.iterrows():
    if index != 4443:
        df_seven = df_seven.append(row)
        
X_wkd = df_seven
X_wkd.index = range(len(X_wkd))

In [57]:
X_wkd['host_is_superhost'] = label_bin.fit_transform(master_wkd.host_is_superhost)
X_wkd['is_location_exact'] = label_bin.fit_transform(master_wkd.is_location_exact)
X_wkd['host_profile_pic'] = label_bin.fit_transform(master_wkd.host_has_profile_pic)
X_wkd['host_identity_verified'] = label_bin.fit_transform(master_wkd.host_identity_verified)
X_wkd['instant_bookable'] = label_bin.fit_transform(master_wkd.instant_bookable)
X_wkd['require_guest_profile_picture'] = label_bin.fit_transform(master_wkd.require_guest_profile_picture)
X_wkd['require_guest_phone_verification'] = label_bin.fit_transform(master_wkd.require_guest_phone_verification)

In [58]:
target_hol = X_hol['price_y']
target_wke = X_wke['price_y']
target_wkd = X_wkd['price_y']

In [59]:
X_hol = X_hol.drop('price_y', axis = 1)
X_wke = X_wke.drop('price_y', axis = 1)
X_wkd = X_wkd.drop('price_y', axis = 1)

In [60]:
target_whole = pd.concat([target_hol, target_wke, target_wkd], axis = 0)
X_whole = pd.concat([X_hol, X_wke, X_wkd], axis = 0)

In [61]:
X_whole = X_whole.fillna(0)

In [62]:
X_whole.shape

(16996, 152)

### Model 4

#### Holidays

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X_hol, target_hol, test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

##### Linear Model

In [64]:
print('R2 for validation is ' + str(r2_est_two(X_train_two, y_train_two, X_val, y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(linear_regression.fit(X_train_two, y_train_two).predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(linear_regression.fit(X_train_two, y_train_two).predict(X_val), y_val)))
print('R2 is ' + str(r2_est_two(X_train, y_train, X_test, y_test)))
print('Mean square error is ' + str(mean_squared_error(linear_regression.fit(X_train, y_train).predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(linear_regression.fit(X_train, y_train).predict(X_test), y_test)))

R2 for validation is 0.664258811108
Mean square error for validation is 9010.65005289
Mean absolute error for validation is 63.6454962801
R2 is 0.625398818979
Mean square error is 9336.71787368
Mean absolute error is 59.9938154053


##### Quadratic Model

In [65]:
create_poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
poly = make_pipeline(create_poly, StandardScaler(), linear_regression)
model = poly.fit(X_train_two, y_train_two)
print('R^2 for validation is ' + str(r2_score(model.predict(X_val), y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(model.predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(model.predict(X_val), y_val)))
model_final = poly.fit(X_train, y_train)
print('R^2 for is ' + str(r2_score(model.predict(X_test), y_test)))
print('Mean square error is ' + str(mean_squared_error(model.predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(model.predict(X_test), y_test)))

R^2 for validation is 0.330959098884
Mean square error for validation is 26586.4579112
Mean absolute error for validation is 107.613344245
R^2 for is 0.282177722496
Mean square error is 32719.593567
Mean absolute error is 125.324642221


#### Weekends

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X_wke, target_wke, test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

##### Linear Model

In [67]:
print('R2 for validation is ' + str(r2_est_two(X_train_two, y_train_two, X_val, y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(linear_regression.fit(X_train_two, y_train_two).predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(linear_regression.fit(X_train_two, y_train_two).predict(X_val), y_val)))
print('R2 is ' + str(r2_est_two(X_train, y_train, X_test, y_test)))
print('Mean square error is ' + str(mean_squared_error(linear_regression.fit(X_train, y_train).predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(linear_regression.fit(X_train, y_train).predict(X_test), y_test)))

R2 for validation is 0.334439008825
Mean square error for validation is 18727.8799723
Mean absolute error for validation is 72.1281829635
R2 is 0.579385359016
Mean square error is 10897.1992709
Mean absolute error is 66.8292724188


##### Quadratic Model

In [68]:
create_poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
poly = make_pipeline(create_poly, StandardScaler(), linear_regression)
model = poly.fit(X_train_two, y_train_two)
print('R^2 for validation is ' + str(r2_score(model.predict(X_val), y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(model.predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(model.predict(X_val), y_val)))
model_final = poly.fit(X_train, y_train)
print('R^2 for is ' + str(r2_score(model.predict(X_test), y_test)))
print('Mean square error is ' + str(mean_squared_error(model.predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(model.predict(X_test), y_test)))

R^2 for validation is 0.00958189752492
Mean square error for validation is 1196438.55344
Mean absolute error for validation is 149.998594867
R^2 for is 0.129135653907
Mean square error is 81673.6402226
Mean absolute error is 150.836054516


#### Weekdays

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X_wkd, target_wkd, test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

##### Linear Model

In [70]:
print('R2 for validation is ' + str(r2_est_two(X_train_two, y_train_two, X_val, y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(linear_regression.fit(X_train_two, y_train_two).predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(linear_regression.fit(X_train_two, y_train_two).predict(X_val), y_val)))
print('R2 is ' + str(r2_est_two(X_train, y_train, X_test, y_test)))
print('Mean square error is ' + str(mean_squared_error(linear_regression.fit(X_train, y_train).predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(linear_regression.fit(X_train, y_train).predict(X_test), y_test)))

R2 for validation is 0.576248397064
Mean square error for validation is 12559.2786654
Mean absolute error for validation is 71.5033793243
R2 is 0.617307937621
Mean square error is 11146.0008107
Mean absolute error is 66.1873838863


##### Quadratic Model

In [71]:
create_poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
poly = make_pipeline(create_poly, StandardScaler(), linear_regression)
model = poly.fit(X_train_two, y_train_two)
print('R^2 for validation is ' + str(r2_score(model.predict(X_val), y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(model.predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(model.predict(X_val), y_val)))
model_final = poly.fit(X_train, y_train)
print('R^2 for is ' + str(r2_score(model.predict(X_test), y_test)))
print('Mean square error is ' + str(mean_squared_error(model.predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(model.predict(X_test), y_test)))

R^2 for validation is 0.181784716548
Mean square error for validation is 46558.4266425
Mean absolute error for validation is 123.286425122
R^2 for is 0.176095000949
Mean square error is 48928.1248248
Mean absolute error is 146.53126753


#### As a whole

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X_whole, target_whole, test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

##### Linear Model

In [73]:
print('R2 for validation is ' + str(r2_est_two(X_train_two, y_train_two, X_val, y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(linear_regression.fit(X_train_two, y_train_two).predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(linear_regression.fit(X_train_two, y_train_two).predict(X_val), y_val)))
print('R2 is ' + str(r2_est_two(X_train, y_train, X_test, y_test)))
print('Mean square error is ' + str(mean_squared_error(linear_regression.fit(X_train, y_train).predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(linear_regression.fit(X_train, y_train).predict(X_test), y_test)))

R2 for validation is 0.637302866951
Mean square error for validation is 9860.6593467
Mean absolute error for validation is 63.4661136148
R2 is 0.609650111887
Mean square error is 10762.8494543
Mean absolute error is 64.7291345333


##### Quadratic Model

In [74]:
create_poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
poly = make_pipeline(create_poly, StandardScaler(), linear_regression)
model = poly.fit(X_train_two, y_train_two)
print('R^2 for validation is ' + str(r2_score(model.predict(X_val), y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(model.predict(X_val), y_val)))
print('Mean absolute error for validation is ' + str(mean_absolute_error(model.predict(X_val), y_val)))
model_final = poly.fit(X_train, y_train)
print('R^2 for is ' + str(r2_score(model.predict(X_test), y_test)))
print('Mean square error is ' + str(mean_squared_error(model.predict(X_test), y_test)))
print('Mean absolute error is ' + str(mean_absolute_error(model.predict(X_test), y_test)))

R^2 for validation is -0.000105905369508
Mean square error for validation is 4.72439347376e+26
Mean absolute error for validation is 7.37650844795e+12
R^2 for is -0.00098538251853
Mean square error is 9.98536414231e+24
Mean absolute error is 635800668735.0


### Impact of Variables for models

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X_hol, target_hol, test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

In [76]:
r2_impact_hol = list()
baseline_hol = r2_est_two(X_train, y_train, X_test, y_test)
for j in range(X_hol.shape[1]):
    selection = [i for i in range(X_hol.shape[1]) if i!=j]
    r2_impact_hol.append((baseline_hol - (r2_est_two(X_train.values[:,selection], y_train, X_test.values[:,selection], y_test)), X_test.columns[j]))
for imp, var in sorted(r2_impact_hol, reverse=True):
    print("%6.3f %s" %(imp, var))

 0.017 accommodates
 0.006 bedrooms
 0.006 cleaning_fee
 0.005 longitude
 0.003 security_deposit
 0.003 host_resp_time_enc
 0.002 minimum_nights
 0.002 has"Elevator in Building"
 0.001 has"Indoor Fireplace"
 0.001 neighborhood_overview_textDigitsPerc
 0.001 description_textWordsPerc
 0.001 has"Suitable for Events"
 0.001 has"Smoking Allowed"
 0.001 has"Free Parking on Premises"
 0.001 number_of_reviews
 0.001 hasTV
 0.001 has
 0.001 require_guest_profile_picture
 0.001 guests_included
 0.001 access_textLength
 0.001 access_diversity
 0.000 review_scores_location
 0.000 host_total_listings_count
 0.000 notes_textDigitsPerc
 0.000 beds
 0.000 reviews_per_month
 0.000 instant_bookable
 0.000 has"translation missing: en.hosting_amenity_49"
 0.000 bed_type_enc
 0.000 require_guest_phone_verification
 0.000 has"Other pet(s)"
 0.000 hasInternet
 0.000 host_acceptance_rate
 0.000 calculated_host_listings_count
 0.000 hasEssentials
 0.000 Topic4
 0.000 has"Pets live on this property"
 0.000 bat

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X_wke, target_wke, test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

In [78]:
r2_impact_wke = list()
baseline_wke = r2_est_two(X_train, y_train, X_test, y_test)
for j in range(X_wke.shape[1]):
    selection = [i for i in range(X_wke.shape[1]) if i!=j]
    r2_impact_wke.append((baseline_wke - (r2_est_two(X_train.values[:,selection], y_train, X_test.values[:,selection], y_test)), X_test.columns[j]))
for imp, var in sorted(r2_impact_wke, reverse=True):
    print("%6.3f %s" %(imp, var))

 0.023 accommodates
 0.006 cleaning_fee
 0.006 longitude
 0.004 security_deposit
 0.002 host_resp_time_enc
 0.002 host_acceptance_rate
 0.002 minimum_nights
 0.002 has"Air Conditioning"
 0.001 calculated_host_listings_count
 0.001 has"Smoking Allowed"
 0.001 availability_90
 0.001 has"Elevator in Building"
 0.001 has"Indoor Fireplace"
 0.001 host_total_listings_count
 0.001 access_textLength
 0.001 reviews_per_month
 0.001 access_diversity
 0.001 has"Laptop Friendly Workspace"
 0.001 zipcode_enc
 0.001 availability_365
 0.001 require_guest_phone_verification
 0.001 has"Suitable for Events"
 0.001 has
 0.001 has"Pets Allowed"
 0.001 availability_60
 0.001 host_response_rate
 0.001 bathrooms
 0.001 review_scores_value
 0.001 latitude
 0.000 Topic11
 0.000 interaction_diversity
 0.000 review_scores_accuracy
 0.000 bed_type_enc
 0.000 Topic10
 0.000 has"Free Parking on Premises"
 0.000 notes_textPuncPerc
 0.000 hasTV
 0.000 has"Pets live on this property"
 0.000 notes_textDigitsPerc
 0.000

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X_wkd, target_wkd, test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

In [80]:
r2_impact_wkd = list()
baseline_wkd = r2_est_two(X_train, y_train, X_test, y_test)
for j in range(X_wkd.shape[1]):
    selection = [i for i in range(X_wkd.shape[1]) if i!=j]
    r2_impact_wkd.append((baseline_wkd - (r2_est_two(X_train.values[:,selection], y_train, X_test.values[:,selection], y_test)), X_test.columns[j]))
for imp, var in sorted(r2_impact_wkd, reverse=True):
    print("%6.3f %s" %(imp, var))

 0.021 accommodates
 0.012 bedrooms
 0.007 longitude
 0.004 has"Elevator in Building"
 0.003 host_resp_time_enc
 0.002 cleaning_fee
 0.002 host_acceptance_rate
 0.002 availability_30
 0.001 has"Air Conditioning"
 0.001 calculated_host_listings_count
 0.001 reviews_per_month
 0.001 beds
 0.001 zipcode_enc
 0.001 require_guest_profile_picture
 0.001 access_diversity
 0.001 availability_90
 0.001 host_response_rate
 0.001 access_textLength
 0.001 Topic11
 0.001 has"Indoor Fireplace"
 0.001 minimum_nights
 0.000 host_total_listings_count
 0.000 has
 0.000 bed_type_enc
 0.000 has"Pets Allowed"
 0.000 instant_bookable
 0.000 has"Family/Kid Friendly"
 0.000 bathrooms
 0.000 require_guest_phone_verification
 0.000 hasHangers
 0.000 has"Safety Card"
 0.000 calendar_updated_enc
 0.000 Topic4
 0.000 security_deposit
 0.000 notes_textDigitsPerc
 0.000 review_scores_accuracy
 0.000 transit_textDigitsPerc
 0.000 notes_textPuncPerc
 0.000 review_scores_checkin
 0.000 has"Pets live on this property"
 

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X_whole, target_whole, test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

In [82]:
r2_impact_whole = list()
baseline_whole = r2_est_two(X_train, y_train, X_test, y_test)
for j in range(X_whole.shape[1]):
    selection = [i for i in range(X_whole.shape[1]) if i!=j]
    r2_impact_whole.append((baseline_whole - (r2_est_two(X_train.values[:,selection], y_train, X_test.values[:,selection], y_test)), X_test.columns[j]))
for imp, var in sorted(r2_impact_whole, reverse=True):
    print("%6.3f %s" %(imp, var))

 0.019 accommodates
 0.006 longitude
 0.005 bedrooms
 0.004 cleaning_fee
 0.003 security_deposit
 0.002 has"Elevator in Building"
 0.001 require_guest_profile_picture
 0.001 has"Air Conditioning"
 0.001 host_resp_time_enc
 0.001 hasPool
 0.001 require_guest_phone_verification
 0.001 minimum_nights
 0.001 availability_90
 0.001 host_acceptance_rate
 0.001 has"Indoor Fireplace"
 0.001 calculated_host_listings_count
 0.001 availability_30
 0.001 access_textLength
 0.001 access_diversity
 0.001 hasInternet
 0.001 has"Free Parking on Premises"
 0.001 reviews_per_month
 0.001 Topic11
 0.001 interaction_diversity
 0.001 interaction_textLength
 0.001 has"Smoking Allowed"
 0.001 guests_included
 0.000 Topic10
 0.000 has"Lock on Bedroom Door"
 0.000 hasHangers
 0.000 hasTV
 0.000 review_scores_communication
 0.000 has"Laptop Friendly Workspace"
 0.000 review_scores_accuracy
 0.000 description_textWordsPerc
 0.000 availability_60
 0.000 has"Buzzer/Wireless Intercom"
 0.000 number_of_reviews
 0.00

### Model 5

#### Holidays

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X_hol, np.log(target_hol), test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

In [84]:
print('R2 for validation is ' + str(r2_est_two(X_train_two, y_train_two, X_val, y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(np.exp(linear_regression.fit(X_train_two, y_train_two).predict(X_val)), np.exp(y_val))))
print('Mean absolute error for validation is ' + str(mean_absolute_error(np.exp(linear_regression.fit(X_train_two, y_train_two).predict(X_val)), np.exp(y_val))))
print('R2 is ' + str(r2_est_two(X_train, y_train, X_test, y_test)))
print('Mean square error is ' + str(mean_squared_error(np.exp(linear_regression.fit(X_train, y_train).predict(X_test)), np.exp(y_test))))
print('Mean absolute error is ' + str(mean_absolute_error(np.exp(linear_regression.fit(X_train, y_train).predict(X_test)), np.exp(y_test))))

R2 for validation is 0.713408608734
Mean square error for validation is 11067.8480221
Mean absolute error for validation is 61.3380258186
R2 is 0.69986984258
Mean square error is 14624.0066227
Mean absolute error is 61.9210474179


#### Weekends

In [85]:
X_train, X_test, y_train, y_test = train_test_split(X_wke, np.log(target_wke), test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

In [86]:
print('R2 for validation is ' + str(r2_est_two(X_train_two, y_train_two, X_val, y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(np.exp(linear_regression.fit(X_train_two, y_train_two).predict(X_val)), np.exp(y_val))))
print('Mean absolute error for validation is ' + str(mean_absolute_error(np.exp(linear_regression.fit(X_train_two, y_train_two).predict(X_val)), np.exp(y_val))))
print('R2 is ' + str(r2_est_two(X_train, y_train, X_test, y_test)))
print('Mean square error is ' + str(mean_squared_error(np.exp(linear_regression.fit(X_train, y_train).predict(X_test)), np.exp(y_test))))
print('Mean absolute error is ' + str(mean_absolute_error(np.exp(linear_regression.fit(X_train, y_train).predict(X_test)), np.exp(y_test))))

R2 for validation is -0.128564923652
Mean square error for validation is 16481.585957
Mean absolute error for validation is 69.1097374351
R2 is 0.651963515773
Mean square error is 18053.5489113
Mean absolute error is 67.9182888195


#### Weekdays

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X_wkd, np.log(target_wkd), test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

In [88]:
print('R2 for validation is ' + str(r2_est_two(X_train_two, y_train_two, X_val, y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(np.exp(linear_regression.fit(X_train_two, y_train_two).predict(X_val)), np.exp(y_val))))
print('Mean absolute error for validation is ' + str(mean_absolute_error(np.exp(linear_regression.fit(X_train_two, y_train_two).predict(X_val)), np.exp(y_val))))
print('R2 is ' + str(r2_est_two(X_train, y_train, X_test, y_test)))
print('Mean square error is ' + str(mean_squared_error(np.exp(linear_regression.fit(X_train, y_train).predict(X_test)), np.exp(y_test))))
print('Mean absolute error is ' + str(mean_absolute_error(np.exp(linear_regression.fit(X_train, y_train).predict(X_test)), np.exp(y_test))))

R2 for validation is 0.635215022373
Mean square error for validation is 15630.5894728
Mean absolute error for validation is 70.7960988664
R2 is 0.68157070758
Mean square error is 19412.1495749
Mean absolute error is 67.5007212331


#### As a whole

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X_whole, np.log(target_whole), test_size=.30, random_state=1)
X_train_two, X_val, y_train_two, y_val = train_test_split(X_train, y_train, test_size=.30, random_state=1)

In [90]:
print('R2 for validation is ' + str(r2_est_two(X_train_two, y_train_two, X_val, y_val)))
print('Mean square error for validation is ' + str(mean_squared_error(np.exp(linear_regression.fit(X_train_two, y_train_two).predict(X_val)), np.exp(y_val))))
print('Mean absolute error for validation is ' + str(mean_absolute_error(np.exp(linear_regression.fit(X_train_two, y_train_two).predict(X_val)), np.exp(y_val))))
print('R2 is ' + str(r2_est_two(X_train, y_train, X_test, y_test)))
print('Mean square error is ' + str(mean_squared_error(np.exp(linear_regression.fit(X_train, y_train).predict(X_test)), np.exp(y_test))))
print('Mean absolute error is ' + str(mean_absolute_error(np.exp(linear_regression.fit(X_train, y_train).predict(X_test)), np.exp(y_test))))

R2 for validation is 0.696978761225
Mean square error for validation is 15485.0512118
Mean absolute error for validation is 65.1062347828
R2 is 0.681936852525
Mean square error is 15603.1353012
Mean absolute error is 65.6321606212
